# Notebook for text extraction on image

The text extraction and analysis is carried out using a variety of tools:  

1. Text extraction from the image using [google-cloud-vision](https://cloud.google.com/vision)  
1. Language detection of the extracted text using [Googletrans](https://py-googletrans.readthedocs.io/en/latest/)  
1. Translation into English or other languages using [Googletrans](https://py-googletrans.readthedocs.io/en/latest/) 
1. Cleaning of the text using [spacy](https://spacy.io/) 
1. Spell-check using [TextBlob](https://textblob.readthedocs.io/en/dev/index.html) 
1. Subjectivity analysis using [TextBlob](https://textblob.readthedocs.io/en/dev/index.html) 
1. Text summarization using [transformers](https://huggingface.co/docs/transformers/index) pipelines
1. Sentiment analysis using [transformers](https://huggingface.co/docs/transformers/index) pipelines 
1. Named entity recognition using [transformers](https://huggingface.co/docs/transformers/index) pipelines 
1. Topic analysis using [BERTopic](https://github.com/MaartenGr/BERTopic) 

The first cell is only run on google colab and installs the [ammico](https://github.com/ssciwr/AMMICO) package.

After that, we can import `ammico` and read in the files given a folder path.

In [1]:
# if running on google colab
# flake8-noqa-cell
import os

if "google.colab" in str(get_ipython()):
    # update python version
    # install setuptools
    # %pip install setuptools==61 -qqq
    # install ammico
    %pip install git+https://github.com/ssciwr/ammico.git -qqq
    # mount google drive for data and API key
    from google.colab import drive

    drive.mount("/content/drive")

In [2]:
import os
import ammico
from ammico import utils as mutils
from ammico import display as mdisplay

We select a subset of image files to try the text extraction on, see the `limit` keyword. The `find_files` function finds image files within a given directory: 

In [3]:
# Here you need to provide the path to your google drive folder
# or local folder containing the images
images = mutils.find_files(
    path="data/",
    limit=10,
)

We need to initialize the main dictionary that contains all information for the images and is updated through each subsequent analysis:

In [4]:
mydict = mutils.initialize_dict(images)

## Google cloud vision API

For this you need an API key and have the app activated in your google console. The first 1000 images per month are free (July 2022).

```
os.environ[
    "GOOGLE_APPLICATION_CREDENTIALS"
] = "your-credentials.json"
```

## Inspect the elements per image
To check the analysis, you can inspect the analyzed elements here. Loading the results takes a moment, so please be patient. If you are sure of what you are doing, you can skip this and directly export a csv file in the step below.
Here, we display the text extraction and translation results provided by the above libraries. Click on the tabs to see the results in the right sidebar. You may need to increment the `port` number if you are already running several notebook instances on the same server.

In [5]:
analysis_explorer = mdisplay.AnalysisExplorer(mydict, identify="text-on-image")
analysis_explorer.run_server(port=8054)

TypeError: __init__() got an unexpected keyword argument 'identify'

## Or directly analyze for further processing
Instead of inspecting each of the images, you can also directly carry out the analysis and export the result into a csv. This may take a while depending on how many images you have loaded. Set the keyword `analyse_text` to `True` if you want the text to be analyzed (spell check, subjectivity, text summary, sentiment, NER).

In [6]:
for key in mydict:
    mydict[key] = ammico.text.TextDetector(
        mydict[key], analyse_text=True
    ).analyse_image()

ReadTimeout: HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10.0)

## Convert to dataframe and write csv
These steps are required to convert the dictionary of dictionarys into a dictionary with lists, that can be converted into a pandas dataframe and exported to a csv file.

In [7]:
outdict = mutils.append_data_to_dict(mydict)
df = mutils.dump_df(outdict)

ValueError: All arrays must be of the same length

Check the dataframe:

In [8]:
df.head(10)

NameError: name 'df' is not defined

Write the csv file - here you should provide a file path and file name for the csv file to be written.

In [9]:
# Write the csv
df.to_csv("./data_out.csv")

NameError: name 'df' is not defined

## Topic analysis
The topic analysis is carried out using [BERTopic](https://maartengr.github.io/BERTopic/index.html) using an embedded model through a [spaCy](https://spacy.io/) pipeline.

BERTopic takes a list of strings as input. The more items in the list, the better for the topic modeling. If the below returns an error for `analyse_topic()`, the reason can be that your dataset is too small.

You can pass which dataframe entry you would like to have analyzed. The default is `text_english`, but you could for example also select `text_summary` or `text_english_correct` setting the keyword `analyze_text` as so:

`ammico.text.PostprocessText(mydict=mydict, analyze_text="text_summary").analyse_topic()`

### Option 1: Use the dictionary as obtained from the above analysis.

In [10]:
# make a list of all the text_english entries per analysed image from the mydict variable as above
topic_model, topic_df, most_frequent_topics = ammico.text.PostprocessText(
    mydict=mydict
).analyse_topic()

Reading data from dict.


ValueError: Please check your provided dictionary -                 no text_english text data found.

### Option 2: Read in a csv
Not to analyse too many images on google Cloud Vision, use the csv output to obtain the text (when rerunning already analysed images).

In [11]:
input_file_path = "data_out.csv"
topic_model, topic_df, most_frequent_topics = ammico.text.PostprocessText(
    use_csv=True, csv_path=input_file_path
).analyse_topic(return_topics=10)

Reading data from df.


ValueError: Please check your provided dataframe -                                 no text_english text data found.

### Access frequent topics
A topic of `-1` stands for an outlier and should be ignored. Topic count is the number of occurence of that topic. The output is structured from most frequent to least frequent topic.

In [12]:
print(topic_df)

NameError: name 'topic_df' is not defined

### Get information for specific topic
The most frequent topics can be accessed through `most_frequent_topics` with the most occuring topics first in the list.

In [13]:
for topic in most_frequent_topics:
    print("Topic:", topic)

NameError: name 'most_frequent_topics' is not defined

### Topic visualization
The topics can also be visualized. Careful: This only works if there is sufficient data (quantity and quality).

In [14]:
topic_model.visualize_topics()

NameError: name 'topic_model' is not defined

### Save the model
The model can be saved for future use.

In [15]:
topic_model.save("misinfo_posts")

NameError: name 'topic_model' is not defined